In [1]:
import pandas as pd
import datetime as dt
import requests_cache

history_length = 5
apikey = '5e225d64b14d31.07865608'
symbol_df_path = 'ITOT_holdings.csv'

In [2]:
from eod_historical_data import get_eod_data
import requests
import os
import datetime as dt
import requests_cache


def get_symbol_data(symbol, history_length, apikey):
    today = dt.datetime.now().strftime('%Y-%m-%d')
    history = (dt.datetime.now() - dt.timedelta(days=365 * history_length)).strftime('%Y-%m-%d')
    df = get_eod_data(symbol, 'US', start=history, end=today, api_key=apikey)
    df.loc[:, 'symbol'] = symbol
    factor = df.Adjusted_close / df.Close
    df.loc[:, 'adj_close'] = df.Adjusted_close
    df.loc[:, 'adj_open'] = df.Open * factor
    df.loc[:, 'adj_high'] = df.High * factor
    df.loc[:, 'adj_low'] = df.Low * factor
    return df


def download_symbols(api_key, history_length, output_dir, symbol_df,
                     indices, failed_indices):
    for index in indices:
        row = symbol_df.iloc[index]
        symbol = row.symbol
        sector = row.sector
        try:            
            print('\r%d: %s' % (index, symbol))
            df = get_symbol_data(symbol, history_length, apikey)
            df.loc[:, 'sector'] = sector
            df.to_parquet('%s/%s.parquet' % (output_dir, symbol))
        except:
            print('failure occured')
            failed_indices.append(index)

            
def download_data(history_length, apikey, symbol_df_path, output_dir='data', limit=1500):
    expire_after = dt.timedelta(days=1)
    session = requests_cache.CachedSession(cache_name='cache', backend='sqlite', expire_after=expire_after)
    symbol_df = pd.read_csv(symbol_df_path).dropna().iloc[:limit]
    failed_indices = []
    download_symbols(apikey, history_length, output_dir, symbol_df,
                     list(range(symbol_df.shape[0])), failed_indices)
    if len(failed_indices) != 0:
        print('retrying on failed symbols')
        failed_indices2 = []
        download_symbols(apikey, history_length, output_dir, symbol_df,
                         failed_indices, failed_indices2)

In [ ]:
download_data(history_length, apikey, symbol_df_path)

In [4]:
import pandas as pd
import os

def combine_data(datadir='data', output_file='all_symbols.parquet'):
    files = os.listdir(datadir)
    dfs = []
    cols = ['adj_close', 'adj_open', 'adj_low', 'adj_high', 'Close', 'symbol', 'sector', 'Date', 'Volume']
    for f in files:
        dfs.append(pd.read_parquet('%s/%s' % (datadir, f)).reset_index()[cols])
    ttl_df = pd.concat(dfs, axis=0, ignore_index=True)
    ttl_df.to_parquet(output_file)
    

In [5]:
combine_data()

In [6]:
df = get_symbol_data('SPY', history_length, apikey)
df.reset_index().to_parquet('market_index.parquet')

In [7]:
import pandas as pd
all_df = pd.read_parquet('all_symbols.parquet')
sector_indices = all_df.groupby(['sector', 'Date']).agg({'adj_close': 'sum'}).reset_index()

In [8]:
sector_indices.to_parquet('sector_index.parquet')